[Reference](https://python.plainenglish.io/python-function-parameter-prototypes-762b9e9e7864)

In [1]:
def add_to_list(some_list = [1]):
   some_list.append(some_list[-1] + 1)
   return some_list

In [2]:
print(add_to_list())
print(add_to_list())
print(add_to_list())

[1, 2]
[1, 2, 3]
[1, 2, 3, 4]


In [3]:
print(add_to_list.__defaults__)

([1, 2, 3, 4],)


In [6]:
def safe_add_to_list(some_list = None):
   if some_list is None: 
       some_list == [1]           
   some_list.append(some_list[-1] + 1)
   return some_list

In [7]:
print(safe_add_to_list())
print(safe_add_to_list())
print(safe_add_to_list())

AttributeError: ignored

In [8]:
from copy import deepcopy

class Prototype(object):
    def __init__(self,argument):
        self._argument = deepcopy(argument)    
        
    def copy(self):
        return deepcopy(self._argument)    
        
    @staticmethod
    def value(item):
        if issubclass(type(item),Prototype):
            return item.copy()
        return item

In [9]:
def some_function(item = Prototype([1])):
    # Returns item or prototype copy.
    v = Prototype.value(item) 
    v.append(v[-1]+1)
    return v

In [10]:
print(some_function())
print(some_function([3]))
print(some_function())

[1, 2]
[3, 4]
[1, 2]


In [15]:
from inspect import signature
from functools import wraps
from copy import deepcopy

def use_defaults_as_prototypes(*param_names):
    def decorator(func):  
        fname = func.__name__
        sig = signature(func)
        no_params = (len(param_names) == 0)        
        
        def copy_fn(val):
            if callable(val):
                return val            
                
            # In case __defaults__ is changed, store a copy.
            val_copy = deepcopy(val) 
            return lambda : deepcopy(val_copy)        
            
        def keep(x): 
            return (x.name in param_names or no_params) \
                and x.default != x.empty        
        items = {
            x.name:copy_fn(x.default) for x in \
               sig.parameters.values() if keep(x)
        }
        err = ", ".join(set(param_names) - set(items.keys()))
        assert err == '',f'{fname} parameter error(s): {err}'                
        
        @wraps(func)
        def wrapper(*args,**kwargs):
                bound = sig.bind(*args,**kwargs)
                for k, copy in items.items():
                    if k not in bound.arguments.keys():
                        bound.arguments[k] = copy()
                bound.apply_defaults()
                return func(*bound.args,**bound.kwargs)             
        return wrapper    
    return decorator

In [16]:
@use_defaults_as_prototypes('item_1')
def some_function(item_1 = [1], item_2 = [2]):
   item_1.append(item_1[-1]+1)
   item_2.append(item_2[-1]+1)
   return {'item_1':item_1, 'item_2':item_2}

In [17]:
print(some_function())
print(some_function([3]))
print(some_function())

{'item_1': [1, 2], 'item_2': [2, 3]}
{'item_1': [3, 4], 'item_2': [2, 3, 4]}
{'item_1': [1, 2], 'item_2': [2, 3, 4, 5]}


In [19]:
class Prototype(object):
    def __init__(self,argument):
        # In case a variable is passed. We want to lock 
        # this value down.
        self._argument = deepcopy(argument)    
        
    def copy(self):
        return deepcopy(self._argument)
        
def generate_defaults_from_prototypes(func):
    sig = signature(func)
    is_prototype = lambda x: isinstance(x.default,Prototype)
    sig_vals = sig.parameters.values()
    items = {x.name:x.default for x in sig_vals if is_prototype(x)}    @wraps(func)
    def wrapper(*args,**kwargs):
        bound = sig.bind(*args, **kwargs)
        for k in items.keys() - bound.arguments.keys():
            bound.arguments[k] = items[k].copy()
        bound.apply_defaults()
        return func(*bound.args, **bound.kwargs)    
        
    return wrapper

In [20]:
@generate_defaults_from_prototypes
def a_function(item_1 = Prototype([1]), item_2 = [2]):
    item_1.append(item_1[-1]+1)
    item_2.append(item_2[-1]+1)
    return {'item_1':item_1,'item_2':item_2}

TypeError: ignored

In [21]:
class Prototype(object):
    def __init__(self,argument):
        self._argument = deepcopy(argument)
    
    def copy(self):
        return deepcopy(self._argument)
        
def generate_defaults_from_prototypes(func):
    sig = signature(func)
    annotations = func.__annotations__.items()
    params = sig.parameters
    def prototype(k,v): return v(params[k].default)
    def not_empty(p): return p.default != p.empty
    
    items = {
        k:prototype(k,v) for k,v in annotations \
        if issubclass(v,Prototype) and not_empty(params[k])
    }    
    
    @wraps (func)
    def wrapper(*args,**kwargs):
        bound = sig.bind(*args,**kwargs)
        for k  in items.keys() - bound.arguments.keys():
            bound.arguments[k] = items[k].copy()
            bound.apply_defaults()
        return func(*bound.args,**bound.kwargs)    
        
    return wrapper

In [22]:
@generate_defaults_from_prototypes 
def some_func(item_1: Prototype = [1], item_2 = [2]):
    item_1.append(item_1[-1]+1)
    item_2.append(item_2[-1]+1)
    return {'item_1':item_1, 'item_2':item_2}